In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("winequality_clear.csv")

In [3]:

df.drop('color', axis=1, inplace=True)

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], shuffle=True,
                                                     stratify=df.iloc[:,-1], train_size=0.8, random_state=1240)

In [6]:
print(x_train.shape, x_test.shape, y_test.shape, y_train.shape)

(5170, 11) (1293, 11) (1293,) (5170,)


In [7]:
y_train.astype(float)
y_test.astype(float)

1354    7.0
5569    6.0
3315    6.0
5016    5.0
1304    6.0
       ... 
1079    5.0
6330    6.0
473     6.0
3705    6.0
2618    5.0
Name: quality, Length: 1293, dtype: float64

# Оценки качества регрессии:

Средняя абсолютная ошибка MAE (абсолютная разница между рассчётным и нужным усредняем) metrics.mean_absolute_error

>>>Средняя квадратичная ошибка MSE (лучше для больших ошибок) metrics.mean_squared_error

Корень из среднеквадратичной ошибки RMSE 

Коэффициент детерминации R2 metrics.r2_score



Линейная регрессия (может быть с несколькими xi в первой степени) LinearRegression (fit_intercept - свободный член(True-False), positive(True-False) - положительные значения)

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
rgr = LinearRegression()
rgr.fit(x_train, y_train)


LinearRegression()

In [10]:
print(rgr.coef_,rgr.intercept_) #выводим коэффициенты регрессии и свободный член

[ 6.28920669e-02 -1.37185711e+00 -1.36758756e-01  4.16652313e-02
 -7.33671005e-01  6.17230947e-03 -2.68989357e-03 -4.76625676e+01
  3.59442054e-01  7.39330340e-01  2.66736219e-01] 48.87217799231189


In [11]:
y_pred = rgr.predict(x_test)
print('MSE', metrics.mean_squared_error(y_test, y_pred), '\nR2: ', metrics.r2_score(y_test, y_pred))

MSE 0.5220310913426291 
R2:  0.3146826262339275


# Полиномная регрессия 

Добавляем просто степени на наши формулы

Функцию таже LinearRegression, но нужно настроить входные данные как полиномные

In [12]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [13]:
pol2 = PolynomialFeatures(degree=2) #делаем вторые степени, добавляет их в выборку
p2_x_train = pol2.fit_transform(x_train) #добавляем вторые степени и перекрестные члены
p2_x_test = pol2.fit_transform(x_test)
p2_x_train.shape

(5170, 78)

In [14]:
rgr2 = LinearRegression() #Полимная регрессия с квадратами
rgr2.fit(p2_x_train, y_train)
y2_pred = rgr2.predict(p2_x_test)
print('MSE', metrics.mean_squared_error(y_test, y2_pred), '\nR2: ', metrics.r2_score(y_test, y2_pred))

MSE 0.49705363579741785 
R2:  0.34747278858532693


In [15]:
pol3 = PolynomialFeatures(degree=3) #делаем третьи степени, добавляет их в выборку
p3_x_train = pol3.fit_transform(x_train) #добавляем вторые степени и перекрестные члены
p3_x_test = pol3.fit_transform(x_test)
rgr3 = LinearRegression() #Полимная регрессия с кубами
rgr3.fit(p3_x_train, y_train)
y3_pred = rgr3.predict(p3_x_test)
print('MSE', metrics.mean_squared_error(y_test, y3_pred), '\nR2: ', metrics.r2_score(y_test, y3_pred)) #Стало ещё хуже чем при второй степени

MSE 0.6180770163397694 
R2:  0.18859446372490207


Можно делать регулиризацию - обычно штраф за сложность модели

# Гребневая регрессия - ридж-регрессия (делает модель более устойчивой)

Тут используется штраф l2 норм это как MSE с членом + g*b^2 

In [16]:
from sklearn.linear_model import Ridge
rgr4 = Ridge() #Регрессия гребневая
rgr4.fit(x_train, y_train)
y4_pred = rgr4.predict(x_test)
print('MSE', metrics.mean_squared_error(y_test, y4_pred), '\nR2: ', metrics.r2_score(y_test, y4_pred)) #Стало ещё хуже чем при второй степени

MSE 0.524556326802552 
R2:  0.31136752151653546


In [17]:
print(rgr.coef_,rgr.intercept_) #коэффициенты обычной регрессии
print(rgr4.coef_,rgr4.intercept_) #коэффициенты ридж горозда меньше изменяются, поспокойней - устойчивей

[ 6.28920669e-02 -1.37185711e+00 -1.36758756e-01  4.16652313e-02
 -7.33671005e-01  6.17230947e-03 -2.68989357e-03 -4.76625676e+01
  3.59442054e-01  7.39330340e-01  2.66736219e-01] 48.87217799231189
[ 0.01392964 -1.47525592 -0.1422238   0.02376406 -0.80444622  0.00617446
 -0.00246063 -0.14736289  0.12270475  0.61419072  0.32370704] 2.3045997742398403


# Модель LASSO - зануляем совсем незначимые коэффициенты чтобы стабилизироваться Lasso

Можно использовать как выбор значимых переменных (shrinkage)

Совмещение ридж и лассо = Elastic Net

Их общие параметры

solver - алгоритм

alpha - величина штрафа

In [18]:
from sklearn.linear_model import Lasso
rgr5= Lasso(alpha=0.0025) #Регрессия лассо
rgr5.fit(x_train, y_train)
y5_pred = rgr5.predict(x_test)
print('MSE', metrics.mean_squared_error(y_test, y5_pred), '\nR2: ', metrics.r2_score(y_test, y5_pred)) #Убили 4 коэффициента, которые незначимы
print(rgr5.coef_,rgr5.intercept_)

MSE 0.5260074862153077 
R2:  0.30946245345802603
[ 0.         -1.32703247 -0.          0.02269952 -0.          0.00632455
 -0.00248516 -0.          0.01220961  0.43850581  0.32810718] 2.5244237455658216


# Дерево решений для регрессии

Минимизируем дисперсию в группах на которые делим

Листья заканчиваются вещественным чилом, так что нужно аккуратно обрезать

In [19]:
from sklearn.tree import DecisionTreeRegressor
rgr6= DecisionTreeRegressor(max_depth=7, random_state=1240) #Регрессия деревом
rgr6.fit(x_train, y_train)
y6_pred = rgr6.predict(x_test)
print('MSE', metrics.mean_squared_error(y_test, y6_pred), '\nR2: ', metrics.r2_score(y_test, y6_pred)) #Так себе


MSE 0.5471189144846919 
R2:  0.2817475743675596


# Случайный лес

Всё как обычно, не то чтобы стабильнл

In [20]:
from sklearn.ensemble import RandomForestRegressor
rgr7= RandomForestRegressor(max_depth=7, n_estimators=1000) #Регрессия лесом
rgr7.fit(x_train, y_train)
y7_pred = rgr7.predict(x_test)
print('MSE', metrics.mean_squared_error(y_test, y7_pred), '\nR2: ', metrics.r2_score(y_test, y7_pred)) 


MSE 0.44928672680797976 
R2:  0.4101807252665909


# Метод k-ближайших соседей



In [21]:
from sklearn.neighbors import KNeighborsRegressor
rgr8 = KNeighborsRegressor(n_neighbors=7)
rgr8.fit(x_train, y_train)
y8_pred = rgr8.predict(x_test)
print('ненормированные данные\nMSE', metrics.mean_squared_error(y_test, y8_pred), '\nR2: ', metrics.r2_score(y_test, y8_pred)) #Ненормированный результат

ненормированные данные
MSE 0.6644411825054848 
R2:  0.12772803427178114


In [22]:
rgr8 = KNeighborsRegressor(n_neighbors=7)
scaler = StandardScaler()
scaler.fit(x_train) #делаем такую функцию для нормирования по x_train
x_train_S = scaler.transform(x_train)
x_test_S = scaler.transform(x_test)
rgr8.fit(x_train_S, y_train)
y8_pred = rgr8.predict(x_test_S)
print('нормированные данные\nMSE', metrics.mean_squared_error(y_test, y8_pred), '\nR2: ', metrics.r2_score(y_test, y8_pred))

нормированные данные
MSE 0.46907208358981634 
R2:  0.38420670001480084


x = np.arange(40)
y = np.zeros_like(x, dtype= float)
k = 100
for _ in range(k):
    x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], shuffle=True,
                                                     stratify=df.iloc[:,-1], train_size=0.8)
    scaler = StandardScaler()
    scaler.fit(x_train) #делаем такую функцию для нормирования по x_train
    x_train_S = scaler.transform(x_train)
    x_test_S = scaler.transform(x_test)
    for i in range(1,41):
        rgr8 = KNeighborsRegressor(n_neighbors=i)
        rgr8.fit(x_train_S, y_train)
        y8_pred = rgr8.predict(x_test_S)
        y[i-1] += metrics.r2_score(y_test, y8_pred)
y /=k

plt.figure(figsize=(5,5))
ax = plt.plot(x, y)

plt.tight_layout()
plt.show()

x = np.arange(5)
y = np.zeros_like(x, dtype= float)
k = 10
for _ in range(k):
    x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], shuffle=True,
                                                     stratify=df.iloc[:,-1], train_size=0.8)
    for i in range(1,6):
        rgr7= RandomForestRegressor(max_depth=23, n_estimators=i*100) #Регрессия лесом
        rgr7.fit(x_train, y_train)
        y7_pred = rgr7.predict(x_test)
        y[i-1] += metrics.r2_score(y_test, y7_pred)
y /=k

plt.figure(figsize=(5,5))
ax = plt.plot(x*100, y)

plt.tight_layout()
plt.show()

In [23]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], shuffle=True,
                                                     stratify=df.iloc[:,-1], train_size=0.8, random_state=1240)
rgr10= RandomForestRegressor(max_depth=23, n_estimators=500) #Регрессия лесом
rgr10.fit(x_train, y_train)
y10_pred = rgr10.predict(x_test)
print('nMSE', metrics.mean_squared_error(y_test, y10_pred), '\nR2: ', metrics.r2_score(y_test, y10_pred))

nMSE 0.3539348919728388 
R2:  0.5353576929160251




x = np.arange(100)
y = np.zeros_like(x, dtype= float)
k = 10
a=0.01
pol3 = PolynomialFeatures(degree=3) #делаем третьи степени, добавляет их в выборку
for _ in range(k):
    x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], shuffle=True,
                                                     stratify=df.iloc[:,-1], train_size=0.8)
    
    p3_x_train = pol3.fit_transform(x_train) #добавляем вторые степени и перекрестные члены
    p3_x_test = pol3.fit_transform(x_test)
    for i in range(1,2):
        rgr5= Lasso(alpha=a*i, max_iter=50000) #Регрессия лассо
        rgr5.fit(p3_x_train, y_train)
        y5_pred = rgr5.predict(p3_x_test)
        y[i-1] += metrics.r2_score(y_test, y5_pred)
y /=k

plt.figure(figsize=(5,5))
ax = plt.plot(x*a, y)

plt.tight_layout()
plt.show()

print('MSE', metrics.mean_squared_error(y_test, y5_pred), '\nR2: ', metrics.r2_score(y_test, y5_pred)) #Убили 4 коэффициента, которые незначимы
print(rgr5.coef_,rgr5.intercept_)

# Метод опорных векторов

In [24]:
from sklearn.svm import SVR
rgr9 = SVR()
rgr9.fit(x_train, y_train)
y9_pred = rgr9.predict(x_test)
print('ненормированные данные\nMSE', metrics.mean_squared_error(y_test, y9_pred), '\nR2: ', metrics.r2_score(y_test, y9_pred))
x_train_S = scaler.transform(x_train)
x_test_S = scaler.transform(x_test)
rgr9.fit(x_train_S, y_train)
y9_pred = rgr9.predict(x_test_S)
print('нормированные данные\nMSE', metrics.mean_squared_error(y_test, y9_pred), '\nR2: ', metrics.r2_score(y_test, y9_pred))

ненормированные данные
MSE 0.643868556287703 
R2:  0.15473557923386405
нормированные данные
MSE 0.4542777200339155 
R2:  0.40362859757380265


# Ансамблевые алгоритмы

Слабый ученик- одиночный вариант обучения (желательно с небольшой предсказательной силой, но не случайная)

Один из них это случайный лес

Стэкинг - собираем результаты разных моделей с одного набора данных, далее собираем результаты - они будут данными для финальной модели (делает предсказание на резльтате промежуточных результатов)

In [25]:
models = [
    KNeighborsRegressor(n_neighbors=7),
    RandomForestRegressor(max_depth=7, n_estimators=50, random_state=1240),
    SVR()
]
meta_x = []
meta_X = []
for model in models:
    model.fit(x_train_S, y_train)
    meta_x.append(model.predict(x_train_S).reshape(-1,1))
    meta_X.append(model.predict(x_test_S).reshape(-1,1))
meta_x = np.hstack(meta_x)
meta_X = np.hstack(meta_X)

In [26]:
print(meta_x) #

[[6.14285714 5.91511976 6.00710305]
 [5.57142857 5.22887182 5.03723699]
 [5.         5.1053893  4.92885718]
 ...
 [6.14285714 6.11212169 5.90948608]
 [6.28571429 6.23799374 6.10148985]
 [5.85714286 5.43705115 5.69182704]]


In [27]:
rgr10 = LinearRegression()
rgr10.fit(meta_x, y_train)
y10_pred = rgr10.predict(meta_X)
print('нормированные данные\nMSE', metrics.mean_squared_error(y_test, y10_pred), '\nR2: ', metrics.r2_score(y_test, y10_pred))

нормированные данные
MSE 0.433638963760836 
R2:  0.43072295743364775


# Бэггинг

Берем несколько выборок и обучаем на можно одинаковых моделях, после делаем опять мета ученика

# Бустинг

Каждый последующий ученик пытается улучшить предыдущего

Адаптивный вариант (Adaboost) - для данных у хорошим результатом делаем малые веса, а для больших ошибок большие. Будет пытаться улучшать только плохие данные 

Градиентный вариант (GradBoost) - делаем функцию потерь от чего-либо (MSE) в зависимости от гиперпараметров системы (градиент MSE -- сумма|y-Y| ), постепенно убиваем коэффициенты или что-то такое для уменьшения функции потерь

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
rgr11 = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, criterion='squared_error', max_depth=7, random_state=1240)
rgr11.fit(x_train, y_train)
y11_pred = rgr11.predict(x_test)
print('нормированные данные\nMSE', metrics.mean_squared_error(y_test, y11_pred), '\nR2: ', metrics.r2_score(y_test, y11_pred))

нормированные данные
MSE 0.3630126896535415 
R2:  0.5234404478145533


XGBoost экстримальный бустинг - используем ещё регуляризацию как в лассо

from xgboost improt XGBRegressor

# Мой код
haha = pd.read_csv('haha.csv', delimiter=';')
haha.columns = haha.iloc[0,:]
haha = haha.drop([0])

haha.head(80)


x_train, x_test, y_train, y_test = train_test_split(haha.iloc[:,1:], haha.iloc[:,0], shuffle=True,
                                                      train_size=0.90)
rgr = GradientBoostingRegressor(n_estimators=30000, learning_rate=0.1, criterion='squared_error', max_depth=7, random_state=1240)
rgr.fit(x_train, y_train)
y_pred = rgr.predict(x_test)
print('нормированные данные\nMSE', metrics.mean_squared_error(y_test, y_pred), '\nR2: ', metrics.r2_score(y_test, y_pred))


haha2 = pd.DataFrame(np.array(y_test).T)
haha2['pred'] = y_pred

haha2.head(20)
# Конец кода